# Valley Bottom Extraction
## Iterate over regional outputs and select input datasets for RVBD tool as well as convert/create necessary datasets.

Create data dictionary to store input parameters for tool.
1. River Network
2. Catchments
3. DEM
4. Wetlands - as feature class or shapefile
5. Output Folder - Directory
6. River Valley Bottom - Output River Valley Bottom Polygon

In [3]:


# Convert wetlands to shapefile
import arcpy
import os
import sys
# Import RVBD modules
import os
from pathlib import Path
import datetime
from data_preparation.RVBD.slope import slope
from data_preparation.RVBD.valleycostdistance import valley
from data_preparation.RVBD.costdistance import costdist

print('imports complete')
print(f'sys paths {sys.path}')

imports complete
sys paths ['C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers\\pydev', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmer

In [4]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [10]:
arcpy.env.overwriteOutput = True
# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# Regions to be processed
rList = ['Kodiak','Prince_William_Sound','Copper_River','Bristol_Bay','Cook_Inlet']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]
#working gdb to save intermediate outputs
rvbd_scratchgdb = r"U:\\GIS\\rvbd_scratch.gdb"
rvbd_scratchfol = r"U:\\GIS\\RVBD"
print (regions)

#Create rvbd scratch folders if they do not already exist
import os, arcpy

rvbd_scratchfol = r"U:\\RVBD_Outputs"
if not arcpy.Exists(rvbd_scratchfol):
    os.mkdir(rvbd_scratchfol)
    print(f'Creating scratch folder {rvbd_scratchfol}')
else:
    print(f'Scratch folder {rvbd_scratchfol} already exists')
print('----------')

if not arcpy.Exists(rvbd_scratchgdb):
    arcpy.CreateFileGDB_management(rvbd_scratchfol,'rvbd_scratch.gdb')
    print(f'Creating scratch gdb {rvbd_scratchgdb}')
else:
    print(f'Scratch folder {rvbd_scratchgdb} already exists')
print('----------')

#Create Regional output folders
for region in regions:
    rname = os.path.basename(region)
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    if not arcpy.Exists(rvbdscratchpath):
        os.mkdir(rvbdscratchpath)
        print (f'{rname} does not have a scratch folder')
        print (f'Creating river valley bottom scratch folder at  {rvbdscratchpath}')
    else:
        print(f'Scratch folder for {rname} already created at {rvbdscratchpath}')

    print('----------')
print(f'All scratch workspaces set')


['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
Creating scratch folder U:\\RVBD_Outputs
----------
Creating scratch gdb U:\\GIS\\rvbd_scratch.gdb
----------
Bristol_Bay does not have a scratch folder
Creating river valley bottom scratch folder at  U:\\RVBD_Outputs\Bristol_Bay_RVBD
----------
Cook_Inlet does not have a scratch folder
Creating river valley bottom scratch folder at  U:\\RVBD_Outputs\Cook_Inlet_RVBD
----------
Copper_River does not have a scratch folder
Creating river valley bottom scratch folder at  U:\\RVBD_Outputs\Copper_River_RVBD
----------
Kodiak does not have a scratch folder
Creating river valley bottom scratch folder at  U:\\RVBD_Outputs\Kodiak_RVBD
----------
Prince_William_Sound does not have a scratch folder
Creating river valley bottom scratch folder at  U:\\RVBD_Outputs\Prince_William_Sound_RVBD
----------
All scratch workspaces set


### Create data dictionary to use as input to RVBD script.
 * Collect and convert datasets as necessary to use as inputs for the RVBD script
 * Create region specific output folders if they do not already exist.


In [13]:
rvbdDict = {}
# Walk through gdbs and set input data sets. Convert wetland raster to shapefile if this has not already been done.
akssf_gdb = r'D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb'
for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, datatype=['RasterDataset','FeatureClass'])
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if 'streams_merge' == filename or 'NHDFlowline_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,strsource)
            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                # Dissolve Watersheds - keep as single part
                wtd_dis = os.path.join(outgdb,rname + '_wtd_dis')
                if not arcpy.Exists(os.path.join(outgdb,rname + 'wtd_dis')):
                    arcpy.Dissolve_management(wtdsource,wtd_dis,None, None, "SINGLE_PART", "DISSOLVE_LINES")
                    # Append dissolved watersheds to Dictionary to use as input for rvbd tool
                append_value(rvbdDict,rname,wtd_dis)
            # Set elev raster
            elif 'elev.tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,elev_path)
            # Set slope raster
            elif 'slope.tif' == filename:
                slope_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,slope_path)
            # Grab wetland raster
            elif 'wetlands.tif' == filename:
                wras_path = os.path.join(dirpath, filename)
                wrasname = filename[:-4]
                wetfcpath = os.path.join(outgdb, wrasname)
                wetfcpath2 = os.path.join(rvbd_scratchgdb, wrasname)
                # Make local copy projected in AKAlbers
                if not arcpy.Exists(wetfcpath):
                    print(f'Converting {wrasname} to feature class...')
                    wetfc_conv = arcpy.RasterToPolygon_conversion(wras_path,wetfcpath2,raster_field='VALUE',
                                                             simplify='NO_SIMPLIFY',
                                                             create_multipart_features='MULTIPLE_OUTER_PART')
                    wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfcpath2,outgdb,wrasname,
                                                                        where_clause='gridcode = 1')
                else:
                    print(f'{wetfcpath} already created')
                    wetfc = wetfcpath
    append_value(rvbdDict,rname,wetfcpath)
    append_value(rvbdDict,rname,rvbdscratchpath)
    rivbottpath = os.path.join(akssf_gdb,rname + '_riv_valbot')
    append_value(rvbdDict,rname,rivbottpath)
    print(f'Stream input = {strsource}')
    print(f'Watersheds input = {wtd_dis}')
    print(f'DEM = {elev_path}')
    print(f'Slope input = {slope_path}')
    print(f'Wetlands input = {wetfc}')
    print('----------')
print('----------')
print('Data Dictionary created')

Bristol_Bay
D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wetlands already created
Stream input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\streams_merge
Watersheds input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\Bristol_Baywtd_dis
DEM = D:\GIS\AKSSF\Bristol_Bay\elev.tif
Slope input = D:\GIS\AKSSF\Bristol_Bay\slope.tif
Wetlands input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wetlands
----------
Cook_Inlet
D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wetlands already created
Stream input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\NHDFlowline_merge
Watersheds input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\Cook_Inletwtd_dis
DEM = D:\GIS\AKSSF\Cook_Inlet\elev.tif
Slope input = D:\GIS\AKSSF\Cook_Inlet\slope.tif
Wetlands input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wetlands
----------
Copper_River
D:\GIS\AKSSF\Copper_River\Copper_River.gdb\wetlands already created
Stream input = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\NHDFlowline_merge
Watersheds input = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\C

In [14]:
rvbdDict

{'Bristol_Bay': ['D:\\GIS\\AKSSF\\Bristol_Bay\\elev.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\slope.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\Bristol_Baywtd_dis',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wetlands',
  'U:\\\\RVBD_Outputs\\Bristol_Bay_RVBD',
  'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\Bristol_Bay_riv_valbot'],
 'Cook_Inlet': ['D:\\GIS\\AKSSF\\Cook_Inlet\\elev.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\slope.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\NHDFlowline_merge',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\Cook_Inletwtd_dis',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wetlands',
  'U:\\\\RVBD_Outputs\\Cook_Inlet_RVBD',
  'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\Cook_Inlet_riv_valbot'],
 'Copper_River': ['D:\\GIS\\AKSSF\\Copper_River\\elev.tif',
  'D:\\GIS\\AKSSF\\Copper_River\\slope.tif',
  'D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\NHDFlowline_merge',
  'D:\

In [15]:
regions

['D:\\GIS\\AKSSF\\Bristol_Bay',
 'D:\\GIS\\AKSSF\\Cook_Inlet',
 'D:\\GIS\\AKSSF\\Copper_River',
 'D:\\GIS\\AKSSF\\Kodiak',
 'D:\\GIS\\AKSSF\\Prince_William_Sound']

### Begin RVBD





In [16]:

# Starting script
start = datetime.datetime.now()
print("Script started at %s" % start)
arcpy.AddMessage ("Script started started at %s" % start)
arcpy.AddMessage ("\n")

# Check out spatial extention
arcpy.CheckOutExtension("Spatial")

# Environments
arcpy.env.overwriteOutput = True

# #try with test data
# # Python inputs
# riv =  r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge" # river network as a shapefile
# cat = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge" # catchments as a shapefile
# dem = r"D:\\GIS\\AKSSF\\Kodiak\\elev.tif"# elevation model as a raster
# wet = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands" # measured valley bottom, wetlands or any wet signature in low areas
# pth = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD" # path to outputs (scratch and cost distance folders)
# outval = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD\\Kodiak_RVBD.gdb\\kod_rvbd" # path to output river valley
# slp_in = r"D:\\GIS\\AKSSF\\Kodiak\\slope.tif"

regions = ['D:\\GIS\\AKSSF\\Kodiak',
           'D:\\GIS\\AKSSF\\Copper_River',
           'D:\\GIS\\AKSSF\\Prince_William_Sound',
           'D:\\GIS\\AKSSF\\Bristol_Bay',
           'D:\\GIS\\AKSSF\\Cook_Inlet',]

# need to iterate over regions and inputs from data dictionary
for region in regions:
    key = os.path.basename(region)
    print(key)
    dem = rvbdDict[key][0]
    slp_in = rvbdDict[key][1]
    riv = rvbdDict[key][2]
    cat = rvbdDict[key][3]
    wet = rvbdDict[key][4]
    pth = rvbdDict[key][5]
    outval = rvbdDict[key][6]

    print(f'riv = {riv}')
    print(f'cat = {cat}')
    print(f'dem = {dem}')
    print(f'wet = {wet}')
    print(f'pth = {pth}')
    print(f'outval = {outval}')
    print(f'slp_in = {slp_in}')
    print('----------')
    # Check to see if scratch geodatabase exists
    s = Path(os.path.join(pth, "outputs.gdb"))
    if s.exists() == False:
        scrg = arcpy.CreateFileGDB_management(pth, "outputs")
        scr = scrg.getOutput(0) # scratch folder
    else:
        scr = os.path.join(pth, "outputs.gdb") #scr = r"in_memory" (if you want to store in memory)

    # Check to see if cost dist folder exists (use code in comments if you want results in a folder instead of gdb)
    c = Path(os.path.join(pth, "costdist.gdb")) # Path(os.path.join(pth, "costdist"))
    if c.exists() == False:
        cstg = arcpy.CreateFileGDB_management(pth, "costdist")  # arcpy.CreateFolder_management(pth, "costdist")
        cst = cstg.getOutput(0) # cost distance folder
    else:
        cst = os.path.join(pth, "costdist.gdb") # os.path.join(pth, "costdist")

    # # Calculate slope and replace 0 slopes with small value - We already have slope rasters so no need for this step
    # slope(riv, cat, dem, scr)
    #
    # Cost distance calculation
    costdist(riv, cat, slp_in, cst, scr)

    # Extract valley
    valley(riv, wet, cst, scr, outval)

# Ending script
end = datetime.datetime.now()
print("Script ended at %s" % end)
arcpy.AddMessage ("Script ended at %s" % end)
arcpy.AddMessage ("\n")
time_elapsed = end - start
print("Time elapsed %s" % (time_elapsed))
arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
arcpy.AddMessage ("\n")
print(".........................................................")
arcpy.AddMessage (".........................................................")
arcpy.AddMessage ("\n")


Script started at 2022-01-09 20:19:34.284118
Kodiak
riv = D:\GIS\AKSSF\Kodiak\Kodiak.gdb\streams_merge
cat = D:\GIS\AKSSF\Kodiak\Kodiak.gdb\Kodiakwtd_dis
dem = D:\GIS\AKSSF\Kodiak\elev.tif
wet = D:\GIS\AKSSF\Kodiak\Kodiak.gdb\wetlands
pth = U:\\RVBD_Outputs\Kodiak_RVBD
outval = D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb\Kodiak_riv_valbot
slp_in = D:\GIS\AKSSF\Kodiak\slope.tif
----------
Cost dist calculation started at 2022-01-09 20:19:34.600457
There is/are 6 intersecting features
Calculating cost distance for OID 1 at 2022-01-09 20:20:11.117162
Completed cost distance for OID 1 at 2022-01-09 20:21:10.751746
.........................................................
Calculating cost distance for OID 2 at 2022-01-09 20:21:10.751746
Completed cost distance for OID 2 at 2022-01-09 20:21:37.066159
.........................................................
Calculating cost distance for OID 3 at 2022-01-09 20:21:37.082127
Completed cost distance for OID 3 at 2022-01-09 20:22:35.462948
.........